In [1]:
import gzip
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import csv
import json
from sklearn.model_selection import train_test_split

## Data Cleaning 

### Convert CSV to Json

In [2]:
# def csv_to_json(csvFilePath, jsonFilePath):
#     data = {}
#     with open(csvFilePath, encoding='utf-8') as csvf:
#         csvReader = csv.DictReader(csvf)
#         for rows in csvReader:
#             key = rows['No']
#             data[key] = rows
#     with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
#         jsonf.write(json.dumps(data, indent=4))
# csv_to_json(r'100k_a.csv', r'100k_a.json')

In [2]:
data = []

df = pd.read_csv('100k_a.csv', header=None)
df = df.rename(columns={0: 'user_id', 1: 'stream id', 2: 'streamer name', 3: 'start time', 4: 'stop time'})
df.head()
# user_id, stream id, streamer name, start time, stop time
# with gzip.open('endomondoHR.json.gz') as f:
#     for l in f:
#         data.append(eval(l))

,user_id,stream id,streamer name,start time,stop time
0,1,33842865744,mithrain,154,156
1,1,33846768288,alptv,166,169
2,1,33886469056,mithrain,587,588
3,1,33887624992,wtcn,589,591
4,1,33890145056,jrokezftw,591,594


In [3]:
df['user_id'].nunique()
df['time spent'] = df['stop time'] - df['start time']

Write here<!-- Although `timestamp_x` and `timestamp_y` are not 'equal', they are actually the same time but represented in different ways. `timestamp_x` is a Unix timestamp (in seconds), while `timestamp_y` is ISO8601 formatted timestamp in UTC. We decided to drop `timestamp_y` as the Unix timestamp is easier to deal with. -->

In [4]:
df

,user_id,stream id,streamer name,start time,stop time,time spent
0,1,33842865744,mithrain,154,156,2
1,1,33846768288,alptv,166,169,3
2,1,33886469056,mithrain,587,588,1
3,1,33887624992,wtcn,589,591,2
4,1,33890145056,jrokezftw,591,594,3
...,...,...,...,...,...,...
3051728,100000,34167930576,mckytv,3582,3583,1
3051729,100000,34168288656,natehill,3582,3583,1
3051730,100000,34169594512,fortnite,3582,3584,2
3051731,100000,34180223072,highdistortion,3709,3711,2


In [5]:
df.isna().sum() # check for NAs

user_id          0
stream id        0
streamer name    0
start time       0
stop time        0
time spent       0
dtype: int64

## EDA

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051733 entries, 0 to 3051732
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   stream id      int64 
 2   streamer name  object
 3   start time     int64 
 4   stop time      int64 
 5   time spent     int64 
dtypes: int64(5), object(1)
memory usage: 139.7+ MB


In [7]:
df.describe()

,user_id,stream id,start time,stop time,time spent
count,3.051733e+06,3.051733e+06,3.051733e+06,3.051733e+06,3.051733e+06
mean,5.032313e+04,3.413002e+10,3.146044e+03,3.149186e+03,3.142054e+00
std,2.889476e+04,1.682242e+08,1.772520e+03,1.772656e+03,4.257966e+00
min,1.000000e+00,3.380153e+10,0.000000e+00,1.000000e+00,1.000000e+00
25%,2.528400e+04,3.398884e+10,1.618000e+03,1.620000e+03,1.000000e+00
50%,5.029900e+04,3.413069e+10,3.174000e+03,3.178000e+03,1.000000e+00
75%,7.545200e+04,3.427399e+10,4.676000e+03,4.679000e+03,3.000000e+00
max,1.000000e+05,3.441642e+10,6.147000e+03,6.148000e+03,9.700000e+01


In [8]:
df

,user_id,stream id,streamer name,start time,stop time,time spent
0,1,33842865744,mithrain,154,156,2
1,1,33846768288,alptv,166,169,3
2,1,33886469056,mithrain,587,588,1
3,1,33887624992,wtcn,589,591,2
4,1,33890145056,jrokezftw,591,594,3
...,...,...,...,...,...,...
3051728,100000,34167930576,mckytv,3582,3583,1
3051729,100000,34168288656,natehill,3582,3583,1
3051730,100000,34169594512,fortnite,3582,3584,2
3051731,100000,34180223072,highdistortion,3709,3711,2


In [11]:
# cont_features = ['ascent', 'descent', 'calories', 'duration', 'distance', 'avg_heart_rate']

In [12]:
# plt.figure(figsize=(10,10))
# for i,feat in enumerate(cont_features):
#     i+=1
#     plt.subplot(2,3,i)
#     sns.histplot(data=merged_df, x=feat, kde=True)

In [13]:
#sns.countplot(data=merged_df, x='sport', palette='deep')

In [14]:
#Shuffle the DF
df = df.sample(frac=1, random_state=42)  
df
train, test = train_test_split(df, test_size=0.3)
train

,user_id,stream id,streamer name,start time,stop time,time spent
2246785,74069,34208258784,esl_csgo,4032,4037,5
792854,26334,34086328048,riotgames,2678,2679,1
1862635,61608,34101914912,s1mple,2834,2838,4
1239022,40865,34281849408,tfue,4764,4765,1
959211,31785,34114183600,mascahs,2987,2988,1
...,...,...,...,...,...,...
101180,3464,33841788464,lincatv,127,128,1
1551545,51188,33969369776,froggen,1445,1446,1
913277,30205,34384097984,loserfruit,5761,5768,7
808211,26845,34366418864,sypherpk,5618,5619,1


In [ ]:
# convert df to list of json
data = df.to_dict(orient='records')
data

In [15]:
streamerPerUser = defaultdict(set)
usersPerStreamer = defaultdict(set)
streamerPopularity = defaultdict(int)
hoursDict = {}

for index, row in train.iterrows():
#     print(row['user_id'], ', ', row['streamer name'])
    user = row['user_id']
    streamer = row['streamer name']
    time = row['time spent']
    streamerPerUser[user].add(streamer)
    usersPerStreamer[streamer].add(user)
    streamerPopularity[streamer] += 1
    hoursDict[(user,streamer)] = time

In [16]:
print(streamerPerUser[1])
usersPerStreamer['berkriptepe']

{'unlostv', 'grimnax', 'kendinemuzisyen', 'ogrencievi', 'elraenn', 'wtcn', 'zeon', 'jahrein', 'towshun', 'raufbaba25', 'mithrain', 'jrokezftw', 'zeusidiouss'}


{92,
 600,
 691,
 719,
 771,
 850,
 1126,
 1656,
 2086,
 2321,
 2504,
 2592,
 2606,
 2610,
 2614,
 3004,
 3805,
 3988,
 5701,
 5762,
 5928,
 5934,
 6140,
 6857,
 6867,
 6915,
 7399,
 7428,
 7582,
 7770,
 7845,
 8009,
 8391,
 8393,
 8400,
 8698,
 8699,
 8996,
 9207,
 9423,
 9429,
 9446,
 9575,
 9676,
 10010,
 10019,
 10157,
 10743,
 11428,
 11666,
 12100,
 12666,
 12807,
 12841,
 12910,
 12976,
 13007,
 13014,
 13029,
 13039,
 13096,
 13582,
 14297,
 14348,
 14475,
 14522,
 16632,
 17374,
 17575,
 18579,
 20355,
 20600,
 20662,
 20671,
 20881,
 21052,
 21814,
 21920,
 22379,
 22381,
 23174,
 23514,
 23800,
 24585,
 25528,
 25542,
 25544,
 25648,
 25649,
 25815,
 26075,
 26119,
 26121,
 26306,
 26381,
 26392,
 26486,
 26758,
 26774,
 26784,
 26877,
 27370,
 27631,
 27680,
 28312,
 28343,
 28546,
 29396,
 30269,
 30304,
 30308,
 31507,
 31775,
 31898,
 32695,
 33002,
 33878,
 34078,
 34450,
 34460,
 34567,
 34937,
 34945,
 35468,
 36953,
 38717,
 41007,
 41393,
 43430,
 43538,
 43565,
 44

In [17]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [18]:
sim = []
threshold = 0.025
count=0
prediction = 0

for index, row in test.iterrows():
    count +=1
    user = row['user_id']
    streamer = row['streamer name']
    s_ = streamerPerUser[user]
    currMax = float('-inf')
    for streamers in s_:
        currScore = Jaccard(usersPerStreamer[streamers],usersPerStreamer[streamer])
        if(currScore > currMax):
            currMax = currScore
        if(currMax == 1):
            prediction +=1
            break
    if(count % 10000 == 0):
        print('here in: ', count)
    sim.append(currMax)

print('here')

# for i in sim:
#     print(i)
#     if i == 1:
#         prediction +=1
        
acc = prediction/len(sim)
print(acc)

here in:  10000
here in:  20000
here in:  30000
here in:  40000
here in:  50000
here in:  60000
here in:  70000
here in:  80000
here in:  90000
here in:  100000
here in:  110000
here in:  120000
here in:  130000
here in:  140000
here in:  150000
here in:  160000
here in:  170000
here in:  180000
here in:  190000
here in:  200000
here in:  210000
here in:  220000
here in:  230000
here in:  240000
here in:  250000
here in:  260000
here in:  270000
here in:  280000
here in:  290000
here in:  300000
here in:  310000
here in:  320000
here in:  330000
here in:  340000
here in:  350000
here in:  360000
here in:  370000
here in:  380000
here in:  390000
here in:  400000
here in:  410000
here in:  420000
here in:  430000
here in:  440000
here in:  450000
here in:  460000
here in:  470000
here in:  480000
here in:  490000
here in:  500000
here in:  510000
here in:  520000
here in:  530000
here in:  540000
here in:  550000
here in:  560000
here in:  570000
here in:  580000
here in:  590000
here i